ref: [DiffAb](https://github.com/luost26/diffab)

## Preparation

First, Runtime -> change runtime type -> GPU

check your python and cuda vesion

In [ ]:
# check your python and cuda version
import os
CUDA_version = os.popen('nvcc --version | egrep -o "V[0-9]+.[0-9]+" | cut -c2-').read().replace('\n','')
print(CUDA_version)
! python --version

11.8
Python 3.10.6


Prepare env (for python=3.10 and cuda=11.8)

In [ ]:
# install condacolab first
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [ ]:
!pip install torch==2.0.0+cu118 torchvision==0.15.0+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install joblib tqdm easydict PyYAML tensorboard biopython==1.78
!conda install -c conda-forge python-lmdb pdbfixer -y
!conda install -c conda-forge -c bioconda abnumber mmseqs2 -y
!sudo apt-get install -y libfftw3-3 #for hdock
# taking 4min

Prepare codes

In [ ]:
import os,shutil,requests,tarfile
if os.getcwd() != '/content':
  os.chdir('/content')
if not os.path.exists('diffab'):
  !git clone https://github.com/luost26/diffab.git
  os.chdir('diffab/trained_models')
  !git clone https://huggingface.co/luost26/DiffAb
  folder_contents = os.listdir('DiffAb')
  for item in folder_contents:
    item_path = os.path.join('DiffAb', item)
    shutil.move(item_path, './')
  os.rmdir('DiffAb')
if os.getcwd() != '/content/diffab/bin':
    os.chdir('/content/diffab/bin')
if not (os.path.exists('createpl')) or not (os.path.exists('hdock')):
    !git clone https://github.com/songjie-guo/diffab-replicate
    shutil.move('./diffab-replicate/createpl','./')
    shutil.move('./diffab-replicate/hdock','./')
    !chmod a+x createpl
    !chmod a+x hdock

Cloning into 'diffab'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 187 (delta 25), reused 22 (delta 22), pack-reused 160
Receiving objects: 100% (187/187), 2.91 MiB | 19.60 MiB/s, done.
Resolving deltas: 100% (63/63), done.
Cloning into 'DiffAb'...
remote: Enumerating objects: 13, done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (13/13), 1.72 KiB | 587.00 KiB/s, done.
Filtering content: 100% (4/4), 231.04 MiB | 62.41 MiB/s, done.
Cloning into 'diffab-replicate'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 842.55 KiB | 7.14 MiB/s, done.
Resolving deltas: 100% (1/1), done.


## Run the model

**Set params first**

NOTE: make sure the output is `params set successfully`. Otherwise, your params are invalid.

In [ ]:
#@title Specify params
#@markdown Do you want to use Antibody-Antigen Complex or Antigen Only? (choose one)
antibody_antigen = False #@param {type:"boolean"}
only_antigen = True #@param {type:"boolean"}

#@markdown Do you want to optimise or sample CDR? (choose one)
sample = True #@param {type:"boolean"}
optimise = False #@param {type:"boolean"}

#@markdown If you want to sample CDR, which part(s)? (you may choose both)
sequence = False #@param {type:"boolean"}
structure = True #@param {type:"boolean"}

#@markdown Sample all CDR simultaneously? or one at a time?  (choose one)
one_at_a_time = True #@param {type:"boolean"}
all_at_a_time = False  #@param {type:"boolean"}

function = ''
model=''

# set params for the python file
if antibody_antigen and only_antigen:
    print('Cannot use "Antibody-Antigen Complex" and "Antigen Only" at the same time.')
elif antibody_antigen and not only_antigen:
    function = 'design_pdb.py'
elif not antibody_antigen and only_antigen:
    function = 'design_dock.py'
else:
    print('Please choose whether to use "Antibody-Antigen Complex" or "Antigen Only".')

# set params for config file
if optimise and sample:
    print('Cannot do "sample" and "optimise" at same time')
elif optimise and not sample:
    model = 'abopt_singlecdr.yml'
    print('You choose to "optimise". NOTE: model can only optimise the sequence and structure of one CDR at a time.')
elif sample and not optimise:
    if sequence and structure:
        if one_at_a_time and not all_at_a_time:
            model = 'codesign_single.yml'
        elif not one_at_a_time and all_at_a_time:
            model = 'codesign_multicdrs.yml'
        else:
            print('Please choose "one_at_a_time" or "all_at_a_time".')
    elif sequence and not structure:
        if one_at_a_time and not all_at_a_time:
            model = 'fixbb.yml'
        else:
            print('Please choose "one_at_a_time". NOTE: Sample only the "sequence" can only do "one_at_a_time".')
    elif not sequence and structure:
        if one_at_a_time and not all_at_a_time:
            model = 'strpred.yml'
        else:
            print('Please choose "one_at_a_time". NOTE: Sample only the "structure" can only do "one_at_a_time".')
    else:
        print('Please choose the part(s) you want to sample.')
else:
    print('Please choose whether to "sample" or "optimise".')

# warning
if function !='' and model !='':
    print("params set successfully.")
    print(f'using function=="{function}", model=="{model}".')
else:
    print('params not valid.')

params set successfully.
using function=="design_dock.py", model=="strpred.yml".


In [ ]:
#@title Input file
#@markdown You need to **input** the correspnding complex/antigen **pdb file** under a folder and **specify  file_path** below.

#@markdown The current folder is `/content/diffab`

#@markdown if you choose to use `Antibody-Antigen Complex`, input the pdb file of the complex:
complex_file = "data/examples/7DK2_AB_C.pdb" #@param {type:"string"}
complex_file = complex_file.strip()
#@markdown if you choose to use `Antigen Only`, input the pdb file of the antigen:
antigen_file = "data/examples/Omicron_RBD.pdb  " #@param {type:"string"}
antigen_file = antigen_file.strip()

config_file = f'./configs/test/{model}'

import os,subprocess
if os.getcwd() != '/content/diffab':
    os.chdir('/content/diffab')

# commanding
if function == 'design_pdb.py':
    if not os.path.exists(complex_file):
        print('Your file path is incorrect or no such file in it.')
    else:
        print("Input file successfully. processing...")
        os.system(f'python {function} {complex_file} --config {config_file}')

if function == 'design_dock.py':
    if not os.path.exists(antigen_file):
        print('Your file path is incorrect or no such file in it.')
    else:
        print("Input file successfully. processing...")
        os.system(f'python {function} --antigen {antigen_file} --config {config_file}')

In [ ]:
#@title Download results
#@markdown All results are stored under folder `/content/diffab/results`
dowload_result = True #@param {type:"boolean"}
if dowload_result:
    from google.colab import files
    !zip -r results.zip results/
    files.download('results.zip')

  adding: results/ (stored 0%)
  adding: results/strpred_Omicron_RBDOmicron_RBDOmicron_RBDOmicron_RBD/ (stored 0%)
  adding: results/strpred_Omicron_RBDOmicron_RBDOmicron_RBDOmicron_RBD/Omicron_RBD_Ab_0003.pdb_2023_07_20__07_39_26/ (stored 0%)
  adding: results/strpred_Omicron_RBDOmicron_RBDOmicron_RBDOmicron_RBD/Omicron_RBD_Ab_0003.pdb_2023_07_20__07_39_26/log.txt (deflated 44%)
  adding: results/strpred_Omicron_RBDOmicron_RBDOmicron_RBDOmicron_RBD/Omicron_RBD_Ab_0003.pdb_2023_07_20__07_39_26/reference.pdb (deflated 79%)
  adding: results/strpred_Omicron_RBD/ (stored 0%)
  adding: results/strpred_Omicron_RBD/Omicron_RBD_Ab_0000.pdb_2023_07_20__05_10_47/ (stored 0%)
  adding: results/strpred_Omicron_RBD/Omicron_RBD_Ab_0000.pdb_2023_07_20__05_10_47/metadata.json (deflated 83%)
  adding: results/strpred_Omicron_RBD/Omicron_RBD_Ab_0000.pdb_2023_07_20__05_10_47/H_CDR2/ (stored 0%)
  adding: results/strpred_Omicron_RBD/Omicron_RBD_Ab_0000.pdb_2023_07_20__05_10_47/H_CDR2/0026.pdb (deflated 7

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>